In [1]:
import urllib2
import json
import datetime
import time
import threading
import timeit
import pandas as pd
import cPickle as pk
from time import strftime,strptime,gmtime
import datetime
import os

import numpy as np
import scipy as sp

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Visualization libraries
# Displays plots in output cell
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing, cross_validation
from sklearn.linear_model import PassiveAggressiveClassifier,PassiveAggressiveRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor
from sklearn.utils import shuffle

In [2]:
def positive_negative (v):
    if v >= 0:
        return 1
    else:
        return 0

In [3]:
def html_downloader(symbol,days=60,days_ago=0):
    #date variables
    dt = datetime.datetime.now()
    UnixTime = int(time.mktime(dt.timetuple()))
    #web variables
    url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*(days+days_ago))+'&period2='+str(UnixTime-86400*days_ago)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
    hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
           'Connection': 'keep-alive'}
    #html data request
    try:
        request = urllib2.Request(url,headers = hdr)
        htmltext = urllib2.urlopen(request)
        data = json.load(htmltext)
        return data
    except:
        pass

In [4]:
def learning_downloader (symbol,days = 10000,shift_days = 10):
    data = html_downloader(symbol,days = days)
    try:     
        #json unpack
        timestamp = data['chart']['result'][0]['timestamp']
        timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
        quote = data['chart']['result'][0]['indicators']['quote'][0]
        sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
        stock_df = pd.DataFrame(quote)
        #index is symbol and timestamp
        stock_df.index = [symbol + str(x) for x in timestamp]

        #moving averages
        stock_df['sma'] = sma50
        stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
        #predict on yesterday's averages
        stock_df['sma'] = stock_df['sma'].shift(1)
        stock_df['vol20'] = stock_df['vol20'].shift(1)
        stock_df = stock_df.dropna()

        #derived columns
        stock_df['c_0'] = stock_df['close']/stock_df['sma']
        stock_df['h_0'] = stock_df['high']/stock_df['sma']
        stock_df['l_0'] = stock_df['low']/stock_df['sma']
        stock_df['o_0'] = stock_df['open']/stock_df['sma']
        stock_df['v_0'] = stock_df['volume']/stock_df['vol20']
        stock_df['r'] = (stock_df['close']-stock_df['open'])/stock_df['open']

        #rolling columns
        for d in list(range(1,shift_days+1)):
            stock_df['c_'+str(d)] = stock_df['c_0'].shift(d)
            stock_df['h_'+str(d)] = stock_df['h_0'].shift(d)
            stock_df['l_'+str(d)] = stock_df['l_0'].shift(d)
            stock_df['o_'+str(d)] = stock_df['o_0'].shift(d)
            stock_df['v_'+str(d)] = stock_df['v_0'].shift(d)
        stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20'], axis=1, inplace=True)
        #do not use current day's intraday data
        stock_df = stock_df.drop(stock_df.index[len(stock_df)-1])
        #clean data
        stock_df = stock_df.dropna()
        stock_df = stock_df[~(stock_df == np.inf).any(axis=1)]
        if len(stock_df) != 0:
            with open('./data/'+symbol+'.p','wb') as f:
                pk.dump(stock_df,f)
    except:
        pass

In [5]:
def stock_predict (symbol,clf,clfR,shift_days=10,days_ago = 0):
    try:
        data = html_downloader(symbol,days_ago = days_ago)
        #json unpack
        timestamp = data['chart']['result'][0]['timestamp']
        timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
        quote = data['chart']['result'][0]['indicators']['quote'][0]
        sma50 = data['chart']['result'][0]['indicators']['sma'][0]['sma']
        stock_df = pd.DataFrame(quote)
        #index is symbol and timestamp
        stock_df.index = [symbol + str(x) for x in timestamp]

        #moving averages
        stock_df['sma'] = sma50
        stock_df['vol20'] = stock_df['volume'].rolling(window=20).mean()
        #predict on yesterday's averages
        stock_df['sma'] = stock_df['sma'].shift(1)
        stock_df['vol20'] = stock_df['vol20'].shift(1)
        stock_df = stock_df.dropna()

        #derived columns
        stock_df['c_0'] = stock_df['close']/stock_df['sma']
        stock_df['h_0'] = stock_df['high']/stock_df['sma']
        stock_df['l_0'] = stock_df['low']/stock_df['sma']
        stock_df['o_0'] = stock_df['open']/stock_df['sma']
        stock_df['v_0'] = stock_df['volume']/stock_df['vol20']

        #rolling columns
        for d in list(range(1,shift_days+1)):
            stock_df['c_'+str(d)] = stock_df['c_0'].shift(d)
            stock_df['h_'+str(d)] = stock_df['h_0'].shift(d)
            stock_df['l_'+str(d)] = stock_df['l_0'].shift(d)
            stock_df['o_'+str(d)] = stock_df['o_0'].shift(d)
            stock_df['v_'+str(d)] = stock_df['v_0'].shift(d)
        stock_df.drop(['close', 'high', 'low', 'open','volume','sma','vol20','c_0','v_0','h_0'], axis=1, inplace=True)
        stock_df = stock_df.dropna()
        stock_df = stock_df[~(stock_df == np.inf).any(axis=1)]
    except:
        pass
    #select last row of downloaded matrix for latest day open price and rolling data
    return '%s will go %s by %s, prediction date: %s'%(symbol, str(clf.predict(stock_df.iloc[[-1]].values.tolist())),str(clfR.predict(stock_df.iloc[[-1]].values.tolist())),timestamp[-1])

In [ ]:
#download learning data from list of stocks
#stock_list = ['AA', 'ABT', 'ABX', 'ADI', 'ADM', 'AET', 'AMD', 'AMR', 'APC', 'AVP', 'AXP', 'BA', 'BAC', 'BAX', 'BBY', 'BK', 'BMC', 'BMY', 'BNI', 'BP', 'CA', 'CAT', 'CI', 'CL', 'COP', 'CVX', 'DD', 'DE', 'DIS', 'DOW', 'EK', 'EMC', 'EMR', 'FNM', 'FRE', 'FRX', 'GE', 'GLW', 'GPS', 'GSK', 'HAL', 'HD', 'HON', 'HPQ', 'HRB', 'IBM', 'IGT', 'JNJ', 'JPM', 'JWN', 'KO', 'KR', 'LLY', 'LOW', 'LTD', 'LUV', 'MCD', 'MDT', 'MMM', 'MO', 'MOT', 'MRK', 'MRO', 'MU', 'MYL', 'NKE', 'NSM', 'NWS', 'OXY', 'PEP', 'PFE', 'PG', 'RSH', 'SLB', 'SLE', 'SLM', 'STJ', 'SYK', 'SYY', 'TGT', 'TJX', 'TMX', 'TXN', 'UN', 'UNH', 'UTX', 'VOD', 'VZ', 'WAG', 'WFC', 'WMB', 'WMT', 'XOM', 'XRX']
#stock_list = ['^GDAXI','^FTSE','^DJI','^FCHI','^N225','^HSI','^AXJO','^GSPC','^IXIC','^TNX','^VIX']
with open("symbols.txt") as symbol_file:
    stock_list = symbol_file.read().split('\n')
threadlist = []
for s in stock_list:
    t = threading.Thread(target = learning_downloader,args=(s,))
    t.start()
    threadlist.append(t)
    #sets top limit of active threads to 50
    while threading.activeCount()>50:
        a=0
    #print threading.activeCount()

for b in threadlist:
    b.join()
print "# of threads: ", len(threadlist)

In [6]:
#load and combine downloaded data for initial learning
ml_df = pd.DataFrame()
directory = './data'
for filename in os.listdir(directory):
    if filename.endswith('.p'):
        stock_df = pk.load(open(os.path.join(directory, filename),'rb'))
        if len(ml_df) == 0:
            #using current bit of data for initial learning
            ml_df = stock_df.tail(60)
        else:
            ml_df = ml_df.append(stock_df.tail(60))
            
#not making decision on intraday close high low and volumes
ml_df = ml_df.drop(['c_0','v_0','h_0'], axis=1)
#regression learning data prep
Xr = np.array(ml_df.drop(['r'],1))
yr = np.array(ml_df['r'])
Xr_train, Xr_test, yr_train, yr_test = cross_validation.train_test_split(Xr,yr,test_size= 0.2)

#classification data prep
ml_df['r'] = ml_df['r'].map(positive_negative)
X = np.array(ml_df.drop(['r'],1))
y = np.array(ml_df['r'])
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size= 0.2)

#Gradiant Boosting Ensemble learning
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train,y_train)
print 'classification score: %s'%(clf.score(X_test, y_test))
clfP = MLPClassifier (solver = 'sgd', warm_start = True).fit(X,y)
with open('init_mplClassifier.p','wb') as f:
    pk.dump(clfP,f)

#GBE regression learning
clfR = GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(Xr_train,yr_train)
print 'regression score: %s'%(clfR.score(Xr_test, yr_test))
clfPR = MLPRegressor (solver = 'sgd', warm_start = True).fit(Xr,yr)
with open('init_mplRegressor.p','wb') as f:
    pk.dump(clfPR,f)

A.p
AA.p
AAAP.p
AABA.p
AAC.p
AAL.p
AAMC.p
AAME.p
AAN.p
AAOI.p
AAON.p
AAP.p
AAPL.p
AAT.p
AAU.p
AAV.p
AAWW.p
AAXJ.p
AAXN.p
AB.p
ABAC.p
ABAX.p
ABB.p
ABBV.p
ABC.p
ABCB.p
ABCD.p
ABDC.p
ABE.p
ABEO.p
ABEV.p
ABG.p
ABIL.p
ABIO.p
ABLX.p
ABM.p
ABMD.p
ABR-PA.p
ABR-PB.p
ABR-PC.p
ABR.p
ABRN.p
ABT.p
ABTX.p
ABUS.p
ABX.p
AC.p
ACAD.p
ACBI.p
ACC.p
ACCO.p
ACER.p
ACET.p
ACFC.p
ACFN.p
ACGL.p
ACGLO.p
ACGLP.p
ACH.p
ACHC.p
ACHN.p
ACHV.p
ACIA.p
ACIU.p
ACIW.p
ACLS.p
ACM.p
ACMR.p
ACN.p
ACNB.p
ACOR.p
ACP.p
ACRE.p
ACRS.p
ACRX.p
ACSF.p
ACST.p
ACT.p
ACTA.p
ACTG.p
ACTL.p
ACU.p
ACUR.p
ACV.p
ACWI.p
ACWX.p
ACXM.p
ACY.p
ADAP.p
ADBE.p
ADC.p
ADES.p
ADI.p
ADM.p
ADMA.p
ADMP.p
ADMS.p
ADNT.p
ADOM.p
ADP.p
ADRD.p
ADRE.p
ADRO.p
ADS.p
ADSK.p
ADSW.p
ADT.p
ADTN.p
ADUS.p
ADVM.p
ADX.p
ADXS.p
AE.p
AEB.p
AED.p
AEE.p
AEG.p
AEGN.p
AEH.p
AEHR.p
AEIS.p
AEK.p
AEL.p
AEM.p
AEMD.p
AEO.p
AEP.p
AER.p
AERI.p
AES.p
AET.p
AETI.p
AEUA.p
AEY.p
AEZS.p
AFAM.p
AFB.p
AFC.p
AFCB.p
AFG.p
AFGE.p
AFGH.p
AFH.p
AFI.p
AFL.p
AFMD.p
AFSI-PA.p
AFSI-PB.p
AFSI-PC.p
AF

In [7]:
#partial fit
# clfP = MLPClassifier (solver = 'sgd', learning_rate='adaptive', warm_start = True, early_stopping = True, learning_rate_init=0.0001, max_iter=2000)
# clfPR = MLPRegressor (solver = 'sgd', learning_rate='adaptive', warm_start = True, early_stopping = True, learning_rate_init=0.0001, max_iter=2000)
clfP = pk.load(open('init_mplClassifier.p','rb'))
clfPR = pk.load(open('init_mplRegressor.p','rb'))

#adjusting hyper parameters for partial fit
clfP.learning_rate='adaptive'
clfP.warm_start = True
clfP.early_stopping = True
clfP.learning_rate_init=0.0001
clfP.max_iter=2000
clfPR.learning_rate='adaptive'
clfPR.warm_start = True
clfPR.early_stopping = True
clfPR.learning_rate_init=0.0001
clfPR.max_iter=2000

directory = './data'
for filename in os.listdir(directory):
    if filename.endswith('.p'):
        stock_df = pk.load(open(os.path.join(directory, filename),'rb'))
        stock_df = stock_df.drop(['c_0','v_0','h_0'], axis=1)
        stock_df = stock_df.dropna()
        stock_df = stock_df[~(stock_df == np.inf).any(axis=1)]
        if len(stock_df) != 0:
            try:
                Xpr = np.array(stock_df.drop(['r'],1))
                ypr = np.array(stock_df['r'])
                Xpr, ypr = shuffle(Xpr,ypr,random_state=0)
                clfPR.partial_fit(Xpr,ypr)
                stock_df['r'] = stock_df['r'].map(positive_negative)
                Xp = np.array(stock_df.drop(['r'],1))
                yp = np.array(stock_df['r'])
                Xp, yp = shuffle(Xp,yp,random_state=0)
                clfP.partial_fit(Xp,yp, classes=np.array([0, 1]))
            except:
                pass
with open('mlpClassifier.p','wb') as f:
    pk.dump(clfP,f)
with open('mlpRegressor.p','wb') as f:
    pk.dump(clfPR,f)
print 'classification score: %s'%(clfP.score(Xp, yp))
print 'regression score: %s'%(clfPR.score(Xpr, ypr))

A.p
AA.p
AAAP.p
AABA.p
AAC.p
AAL.p
AAMC.p
AAME.p
AAN.p
AAOI.p
AAON.p
AAP.p
AAPL.p
AAT.p
AAU.p
AAV.p
AAWW.p
AAXJ.p
AAXN.p
AB.p
ABAC.p
ABAX.p
ABB.p
ABBV.p
ABC.p
ABCB.p
ABCD.p
ABDC.p
ABE.p
ABEO.p
ABEV.p
ABG.p
ABIL.p
ABIO.p
ABLX.p
ABM.p
ABMD.p
ABR-PA.p
ABR-PB.p
ABR-PC.p
ABR.p
ABRN.p
ABT.p
ABTX.p
ABUS.p
ABX.p
AC.p
ACAD.p
ACBI.p
ACC.p
ACCO.p
ACER.p
ACET.p
ACFC.p
ACFN.p
ACGL.p
ACGLO.p
ACGLP.p
ACH.p
ACHC.p
ACHN.p
ACHV.p
ACIA.p
ACIU.p
ACIW.p
ACLS.p
ACM.p
ACMR.p
ACN.p
ACNB.p
ACOR.p
ACP.p
ACRE.p
ACRS.p
ACRX.p
ACSF.p
ACST.p
ACT.p
ACTA.p
ACTG.p
ACTL.p
ACU.p
ACUR.p
ACV.p
ACWI.p
ACWX.p
ACXM.p
ACY.p
ADAP.p
ADBE.p
ADC.p
ADES.p
ADI.p
ADM.p
ADMA.p
ADMP.p
ADMS.p
ADNT.p
ADOM.p
ADP.p
ADRD.p
ADRE.p
ADRO.p
ADS.p
ADSK.p
ADSW.p
ADT.p
ADTN.p
ADUS.p
ADVM.p
ADX.p
ADXS.p
AE.p
AEB.p
AED.p
AEE.p
AEG.p
AEGN.p
AEH.p
AEHR.p
AEIS.p
AEK.p
AEL.p
AEM.p
AEMD.p
AEO.p
AEP.p
AER.p
AERI.p
AES.p
AET.p
AETI.p
AEUA.p
AEY.p
AEZS.p
AFAM.p
AFB.p
AFC.p
AFCB.p
AFG.p
AFGE.p
AFGH.p
AFH.p
AFI.p
AFL.p
AFMD.p
AFSI-PA.p
AFSI-PB.p
AFSI-PC.p
AF

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [8]:
#prediction testing
dt = datetime.datetime.now()
UnixTime = int(time.mktime(dt.timetuple()))
symbol = 'fas'

clf = pk.load(open('mlpClassifier.p','rb'))
clfR = pk.load(open('mlpRegressor.p','rb'))
for d in list(range(0,3))[::-1]:
    print stock_predict(symbol,clf,clfR,days_ago=d)

#web variables
url = 'https://query1.finance.yahoo.com/v8/finance/chart/'+symbol+'?period1='+str(UnixTime-86400*5)+'&period2='+str(UnixTime)+'&interval=1d&indicators=quote%7Csma~50&includePrePost=true&events=div%7Csplit%7Cearn&lang=en-CA&region=CA&corsDomain=ca.finance.yahoo.com'
hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36',
'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
'Connection': 'keep-alive'}
#html data request
request = urllib2.Request(url,headers = hdr)
htmltext = urllib2.urlopen(request)

data = json.load(htmltext)
#json unpack
timestamp = data['chart']['result'][0]['timestamp']
timestamp = [datetime.datetime.fromtimestamp(x).strftime('%Y%m%d') for x in timestamp]
quote = data['chart']['result'][0]['indicators']['quote'][0]
stock_df = pd.DataFrame(quote)
#index is symbol and timestamp
stock_df.index = [symbol + str(x) for x in timestamp]
stock_df = stock_df.dropna()
stock_df['r'] = (stock_df['close']-stock_df['open'])/stock_df['open']
print stock_df

fas will go [1] by [ nan], prediction date: 20180329
fas will go [1] by [ nan], prediction date: 20180402
fas will go [1] by [ nan], prediction date: 20180402
                 close       high        low       open   volume         r
fas20180328  62.049999  63.470001  60.189999  61.959999  2010700  0.001453
fas20180329  64.279999  65.300003  62.139999  63.040001  1831400  0.019670
fas20180402  60.369999  64.400002  58.240002  63.939999  2793400 -0.055834
